## Load the tips dataset.

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import prepare as p

from pydataset import data
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression

In [2]:
# Load the tips dataset
df = sns.load_dataset("tips")

# Let's take a quick look at the dataset
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df['size'].value_counts()

2    156
3     38
4     37
5      5
1      4
6      4
Name: size, dtype: int64

## Create a column named price_per_person. This should be the total bill divided by the party size.

In [4]:
df['price_per_person'] = df['total_bill'] / df['size']

df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


## Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

total_bill, size, price_per_person

## Use Select K Best to select the top 2 features for predicting tip amount. What are they?

In [5]:
train, val, test = p.split_data(df)

train.shape, val.shape, test.shape

((170, 8), (37, 8), (37, 8))

In [6]:
train = pd.get_dummies(train, columns = ['sex', 'smoker', 'time'], drop_first = True)
train = pd.get_dummies(train, columns = ['day'])

train.head()

,total_bill,tip,size,price_per_person,sex_Female,smoker_No,time_Dinner,day_Thur,day_Fri,day_Sat,day_Sun
234,15.53,3.00,2,7.7650,0,0,1,0,0,1,0
227,20.45,3.00,4,5.1125,0,1,1,0,0,1,0
180,34.65,3.68,4,8.6625,0,0,1,0,0,0,1
5,25.29,4.71,4,6.3225,0,1,1,0,0,0,1
56,38.01,3.00,4,9.5025,0,0,1,0,0,1,0


In [8]:
mms = MinMaxScaler()

to_scale = train.drop(columns = ['tip']).columns

train[to_scale] = mms.fit_transform(train[to_scale])

train.head()

,total_bill,tip,size,price_per_person,sex_Female,smoker_No,time_Dinner,day_Thur,day_Fri,day_Sat,day_Sun
234,0.217044,3.00,0.2,0.281034,0.0,0.0,1.0,0.0,0.0,1.0,0.0
227,0.326232,3.00,0.6,0.128592,0.0,1.0,1.0,0.0,0.0,1.0,0.0
180,0.641367,3.68,0.6,0.332615,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,0.433644,4.71,0.6,0.198132,0.0,1.0,1.0,0.0,0.0,0.0,1.0
56,0.715934,3.00,0.6,0.380891,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [9]:
X = train.drop(columns = ['tip'])
y = train.tip

In [18]:
skb = SelectKBest(f_regression, k = 3)

skb.fit(X,y)

SelectKBest(k=3, score_func=<function f_regression at 0x152c69260>)

In [19]:
skb_mask = skb.get_support()

X.columns[skb_mask]

Index(['total_bill', 'size', 'price_per_person'], dtype='object')

Top 3: total_bill, size, price_per_person

## Use Recursive Feature Elimination to select the top 2 features for tip amount. What are they?

In [20]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select = 3)

rfe.fit(X, y)

RFE(estimator=LinearRegression(), n_features_to_select=3)

In [21]:
rfe_mask = rfe.get_support()

X.columns[rfe_mask]

Index(['total_bill', 'price_per_person', 'smoker_No'], dtype='object')

TOP 3: total_bill, price_per_person, smoker_No

## Why do you think Select K Best and Recursive Feature Elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

They give different answers because they are two different approaches in identifying feature importance. The order of importance stays consistent but additional features will be added to the end.

## Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [56]:
def select_kbest(X, y, k):
    
    skb = SelectKBest(f_regression, k = k)

    skb.fit(X,y)
    
    skb_mask = skb.get_support()

    return X.columns[skb_mask]
    

In [57]:
select_kbest(X, y, k = 2)

Index(['Examination', 'Education'], dtype='object')

## Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top n features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [39]:
def ref(X, y, n):
    
    lm = LinearRegression()

    rfe = RFE(lm, n_features_to_select = n)

    rfe.fit(X, y)
    
    rfe_mask = rfe.get_support()

    return X.columns[rfe_mask]

In [40]:
ref(X, y, 2)

Index(['total_bill', 'price_per_person'], dtype='object')

## Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both Select K Best and Recursive Feature Elimination (use the functions you just built to help you out).

In [51]:
df = data('swiss')

df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [46]:
#train, val, test = p.split_data(df)

#train.shape, val.shape, test.shape

((32, 6), (7, 6), (8, 6))

In [47]:
X = df.drop(columns = ['Fertility'])
y = df.Fertility

In [48]:
select_kbest(X, y, 3)

Index(['Examination', 'Education', 'Catholic'], dtype='object')

In [49]:
ref(X, y, 3)

Index(['Examination', 'Education', 'Infant.Mortality'], dtype='object')